# Import Modules

In [88]:
# モジュールをインポート
from PyAstronomy.pyasl import SWEETCat
import numpy as np
import pandas as pd
import openpyxl
import xlsxwriter
import datetime
import warnings

# pandasの行と列の最大出力数を指定
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 1000)

# 警告を非表示
warnings.filterwarnings('ignore')

# Update

## Read Files

In [89]:
# Excelファイルの読み出し
directory_name = '/Users/shohei/Desktop/Shohei/Fresnel/Formation/Data'
file_name = f'{directory_name}/dataset_new.xlsx'
sheet_name = '2018-10-25'
excel_book = pd.ExcelFile(file_name)
df_old = excel_book.parse(sheet_name)
print(f'Last update of dataset: {sheet_name}')

Last update of dataset: 2018-10-25


In [51]:
# SWEET-CATカタログのデータの読み出し
sweet_cat = SWEETCat().data

# データが2つあるものの古い方を削除
#sweet_cat = sweet_cat.drop(166)

In [52]:
# exoplanet.euのデータの読み出し
eu = pd.read_csv('/Users/shohei/Desktop/Shohei/Fresnel/Formation/Data/exoplanet.eu_catalog.csv')
print('Last update of exoplanet.eu: 2018-10-25')

Last update of exoplanet.eu: 2018-10-25


## Select Columns

In [53]:
## 必要なカラムを取得
#df_old = df_old[['# name',
#                 'star_name',
#                 '[Fe/H]_sweet', '[Fe/H]_err',
#                 'M*', 'M*_err',
#                 'PER', 'UPER',
#                 'ECC', 'UECC',
#                 'K', 'UK',
#                 'SEMI', 'USEMI',
#                 'Mp', 'UMp',
#                 'Ump_delta_K',
#                 'Ump_delta_e',
#                 'Ump_delta_P',
#                 'Ump_delta_M*',
#                 'Term_year',
#                 'Accuracy'
#                 ]]

In [54]:
# カラム名を変換
#df_old = df_old.rename(columns={'# name': 'planet',
#                                'star_name': 'star',
#                                '[Fe/H]_sweet': 'Fe/H', '[Fe/H]_err': 'dFe/H',
#                                'M*': 'Ms', 'M*_err': 'dMs',
#                                'PER': 'P', 'UPER': 'dP',
#                                'ECC': 'e', 'UECC': 'de',
#                                'K': 'K', 'UK': 'dK',
#                                'SEMI': 'a', 'USEMI': 'da',
#                                'Mp': 'Mp', 'UMp': 'dMp',
#                                'Ump_delta_K': 'DK',
#                                'Ump_delta_e': 'De',
#                                'Ump_delta_P': 'DP',
#                                'Ump_delta_M*': 'DMs',
#                                'Term_year': 'Term',
#                                'Accuracy': 'RMS',
#                                })

sweet_cat = sweet_cat.rename(columns={'mass': 'Ms_sweet', 'ermass': 'dMs_sweet',
                                      'metal': 'Fe/H_sweet', 'ermetal': 'dFe/H_sweet',
                                      'author': 'reference_sweet'
                                     })

eu = eu.rename(columns={'# name': 'planet',
                        'orbital_period': 'P_eu',
                        'eccentricity': 'e_eu',
                        'k': 'K_eu',
                        'star_metallicity': 'Fe/H_eu',
                        'star_mass': 'Ms_eu'
                       })                        

In [55]:
# exoplanet.euの誤差データのカラムを作成
eu['dP_eu'] = (eu['orbital_period_error_min']+eu['orbital_period_error_max'])*0.5
eu['de_eu'] = (eu['eccentricity_error_min']+eu['eccentricity_error_max'])*0.5
eu['dK_eu'] = (eu['k_error_min']+eu['k_error_max'])*0.5
eu['dFe/H_eu'] = (eu['star_metallicity_error_min']+eu['star_metallicity_error_max'])*0.5
eu['dMs_eu'] = (eu['star_mass_error_min']+eu['star_mass_error_max'])*0.5

In [82]:
# SWEET-Catカタログから主星の質量と金属量のデータ、およびその誤差データを取得
cols_sweet = ['star', 'Ms_sweet', 'dMs_sweet', 'Fe/H_sweet', 'dFe/H_sweet', 'reference_sweet']
cols_eu = ['planet', 'P_eu', 'dP_eu', 'e_eu', 'de_eu', 'K_eu', 'dK_eu', 'Fe/H_eu', 'dFe/H_eu', 'Ms_eu', 'dMs_eu']
df_new = pd.merge(df_old, sweet_cat[cols_sweet], on='star', how='left')
df_new = pd.merge(df_new, eu[cols_eu], on='planet', how='left')

## Revise Data

In [83]:
# データがない行を削除
df_new = df_new.dropna(how='all')

In [84]:
# datasetの値を最新のデータに更新
cols_old = ['Fe/H', 'dFe/H', 'Ms', 'dMs', 'P', 'dP', 'e', 'de', 'K', 'dK', 'reference']
cols_new = ['Fe/H_sweet', 'dFe/H_sweet', 'Ms_sweet', 'dMs_sweet', 'P_eu', 'dP_eu', 'e_eu', 'de_eu', 'K_eu', 'dK_eu', 'reference_sweet']
for col_old, col_new in zip(cols_old, cols_new):
    df_new.loc[df_new[col_new].isnull()==False, col_old] = df_new[df_new[col_new].isnull()==False][col_new]

In [86]:
# 要らないカラムを削除
drop_cols = ['reference_sweet', 'P_eu', 'dP_eu', 'e_eu', 'de_eu', 'K_eu', 'dK_eu']
df_new = df_new.drop(drop_cols, axis=1)

## Save File

In [106]:
# updateしたDFをExcelファイルの新たなsheetに追加
today = datetime.date.today().isoformat()
with pd.ExcelWriter(file_name, engine='openpyxl') as writer:
    writer.book = openpyxl.load_workbook(file_name)
    df_new.to_excel(writer, sheet_name=f'{today}', index=False)

In [87]:
## DFをExcelファイルとして保存
#df_new.to_excel('/Users/shohei/Desktop/Shohei/Fresnel/Formation/Data/dataset_new_v2.xlsx', sheet_name=f'{today}', index=False)